# Imports

In [1]:
import os
os.chdir("D:\PulpitE\FPL_ML")

In [2]:
import pandas as pd
import time
from vaastav.fbref import *
from src.match_names import neutralize_name

# Getting raw data

In [3]:
# only Mo Salah
only_Salah = False

In [4]:
def get_url( url ):
    df = pd.read_html( url, header=1)[0]
    return df

In [5]:
def get_shorter_season(season):
    res = season.split("-")
    return res[0] + "-" + res[1][2:]

In [6]:
def get_logs_for_players_in_season(players, season, players_names=[]):
    logs = pd.DataFrame({'A' : []})
    for id, player in players:
        player_name = neutralize_name(player)
        if only_Salah and player_name != "Mohamed-Salah": 
            continue
        print("Getting data for " + player_name)
        url = 'https://fbref.com/en/players/' + str(id) + '/matchlogs/' + season + '/summary/' + player_name + '-Match-Logs'
        try:
            new_player_df = get_url(url)
            new_player_df["Name"] = player_name
            new_player_df["Season"] = get_shorter_season(season)


            # dropping NaN rows
            new_player_df = new_player_df[new_player_df['Date'].notna()]

            # only Premier League
            # new_player_df = new_player_df[new_player_df.Comp == "Premier League"]

            # # only last 15 matches
            # new_player_df = new_player_df[-15:]

            if not logs.empty:
                logs = pd.concat([logs, new_player_df])
            else:
                logs = new_player_df
        except:
            print("Not found", player_name, season, id, url)
        
        # to not get timeouted
        time.sleep(3.2)
    return logs

In [7]:
tables = get_data("https://fbref.com/en/comps/9/wages/Premier-League-Wages")

In [8]:
# tables[1]

In [9]:
players = get_epl_players()

In [10]:
len(players)

532

In [11]:
# players

In [12]:
# players['774cf58b'].data

In [13]:
def modify_df(df):
    df[['WDL', 'GoalsTeams']] = df['Result'].str.split(' ', expand=True)
    df[['Team Score', 'Opp Score']] = df['GoalsTeams'].str.split('–', expand=True)
    df['Team Score'] = df['Team Score'].astype(int)
    df['Opp Score'] = df['Opp Score'].astype(int)
    df = df.drop(['Result'], axis=1)
    
    df = df.replace("On matchday squad, but did not play", 0)
    df["Min"] = df["Min"].astype(float)
    df["Gls"] = df["Gls"].astype(float)
    df["Ast"] = df["Ast"].astype(float)
    
    df["CS"] = (df["Opp Score"] == 0).astype(float)
    df["Was Home"] = (df["Venue"] == "Home").astype(float)
    df["GW"] = df['Round'].str.split(' ', expand=True)[1]
    
    df = df.sort_values(by=['Name', 'Date'], ascending=[True, True])
    
    return df

# 2021-22

In [14]:
%%time
# player_names = ["Mohamed-Salah", "Miguel-Almiron", "Bukayo-Saka", "Kevin-De-Bruyne", "Wilfried-Zaha", "Mason-Mount", "Bruno-Fernandes"]
logs_21_22 = get_logs_for_players_in_season(players, '2021-2022', None)

Getting data for Kevin-De-Bruyne
Getting data for Erling-Haaland
Getting data for Casemiro
Getting data for Jadon-Sancho
Getting data for Mohamed-Salah
Getting data for Raphael-Varane
Getting data for Raheem-Sterling
Getting data for Romelu-Lukaku
Getting data for Marcus-Rashford
Getting data for Jack-Grealish
Getting data for Kai-Havertz
Getting data for Gabriel-Jesus
Getting data for Anthony-Martial
Getting data for Joao-Cancelo
Getting data for John-Stones
Getting data for Reece-James
Getting data for Declan-Rice
Getting data for Bruno-Fernandes
Getting data for Phil-Foden
Getting data for Virgil-van-Dijk
Getting data for Rodri
Getting data for Sandro-Tonali
Getting data for Thomas-Partey
Getting data for Antony
Getting data for Mason-Mount
Getting data for Tanguy-Ndombele
Getting data for Thiago-Alcantara
Getting data for Josko-Gvardiol
Getting data for Ben-Chilwell
Getting data for Wesley-Fofana
Getting data for Bukayo-Saka
Getting data for Christopher-Nkunku
Getting data for Will

In [15]:
logs_21_22 = logs_21_22.reset_index()

In [16]:
logs_21_22.shape

(21707, 48)

In [ ]:
logs_21_22 = modify_df(logs_21_22)
logs_21_22.to_csv("data/logs_21_22.csv")

# 2022-23

In [ ]:
%%time
# player_names = ["Mohamed-Salah", "Miguel-Almiron", "Bukayo-Saka", "Kevin-De-Bruyne", "Wilfried-Zaha", "Mason-Mount", "Bruno-Fernandes"]
logs_22_23 = get_logs_for_players_in_season(players, '2022-2023', None)

In [ ]:
logs_22_23.shape

In [ ]:
logs_22_23 = modify_df(logs_22_23)
logs_22_23.to_csv("data/logs_22_23.csv")

# Merged logs

In [ ]:
# merging
logs = pd.concat([logs_21_22, logs_22_23]).reset_index()

In [ ]:
logs.shape

In [ ]:
logs.head(10)[["Date", "Round", "Name"]]

In [ ]:
logs.columns

In [ ]:
logs.to_csv("data/logs_all.csv")

# Features check

In [ ]:
info = ["Date", "Day", "GW", "Was Home"]
features = ["Min", "Gls", "Sh", "SoT", "xG", "npxG", "xAG", "CS"]

In [ ]:
logs[info + features]

In [ ]:
logs[logs["Name"] == "Mohamed-Salah"]